In [1]:
import pandas as pd
import os

In [60]:
class company_prerocessing():
#     def __init__()
    # 파일 열기
    def open_df(folder_path):
        df = pd.read_csv(folder_path)
        return df

    # 재무 상태표
    def bs_preprocessing(df_bs, cmp_list):
        
        # 행 3번째가 숫자면 0, 2, 3, 4 행만 가져오기
        if df_bs.columns[2].isdigit() == True:
            try:
                df_bs_1 = df_bs.iloc[:, [0, 2, 3, 4]]
            except:
                pass
            
        # 행 6번째가 숫자면 1, 6, 7, 8 행만 가져오기
        elif df_bs.columns[6].isdigit() == True:
            df_bs_1 = df_bs.iloc[:, [1, 6, 7, 8]]
            
        # 행 7번째가 숫자면 1, 7, 8, 9 행만 가져오기
        elif df_bs.columns[7].isdigit() == True:
            df_bs_1 = df_bs.iloc[:, [1, 7, 8, 9]]
        
        # 행 8번째가 숫자면 1, 8, 9, 10 행만 가져오기
        elif df_bs.columns[8].isdigit() == True:
            df_bs_1 = df_bs.iloc[:, [1, 8, 9, 10]]
            
        # 위에 모든 조건이 맞지 않다면 0:4 가져오기
        else:
            df_bs_1 = df_bs.iloc[:, 0:4]
            
        # 컬럼 이름 지정
        df_bs_1.columns = ['', df_bs_1.columns[1], df_bs_1.columns[2], df_bs_1.columns[3]]
        # 데이터 프레임 전치
        df_bs_1 = df_bs_1.T
        
        # 행 이름을 0번째 줄로 바꿈
        df_bs_1 = df_bs_1.rename(columns=df_bs_1.iloc[0])
        # 0번째 열 제거
        df_bs_1 = df_bs_1.drop(df_bs_1.index[0])
        # 'label_ko' 행 제거
        df_bs_1.drop('label_ko', inplace=True, axis =1)
        
        # 인덱스 리셋, index 행이름 바꾸기
        df_bs_1 = df_bs_1.reset_index().rename(columns={'index': 'date'})

        # company  행 추가 후 회사 이름 넣기
        df_bs_1['company'] = cmp_list
        print('bs_preprocessing complete')

        return df_bs_1

    # 포괄손익 계산서
    def cis_preprocessing(df_cis, cmp_list):
        # 7번째 행에 01-20이 포함되었을때
        if '01-20' in df_cis.columns[6]:
            df_cis_1 = df_cis.iloc[:, [1, 6, 7]]
        # 7번째 행에 01-20에 포함되지 않았을때
        else:
            df_cis_1 = df_cis.iloc[:, [1, 7, 8]]
        
        # 컬럼 이름 지정
        df_cis_1.columns = ['', df_cis_1.columns[1], df_cis_1.columns[2]]
        
        # 데이터 프레임 전치
        df_cis_1 = df_cis_1.T
        
        # 행 이름을 0번째 줄로 바꿈
        df_cis_1 = df_cis_1.rename(columns=df_cis_1.iloc[0])
        # 0번째 열 제거
        df_cis_1 = df_cis_1.drop(df_cis_1.index[0])
        # 'label_ko' 행 제거
        df_cis_1.drop('label_ko', inplace=True, axis =1)
        
        # 인덱스 리셋, index 행이름 바꾸기
        df_cis_1 = df_cis_1.reset_index().rename(columns={'index': 'date'})
        
        # company  행 추가 후 회사 이름 넣기
        df_cis_1['company'] = cmp_list
        # date의 9번째 줄 부터 끝까지 바꾸기
        df_cis_1['date'] = df_cis_1['date'].str[9:]
        print('cis_preprocessing complete')

        return df_cis_1

    # 데이터 프레임 저장
    def save_df(folder_path, df, cmp_list):
        df.to_csv(folder_path + cmp_list + '.csv', index=False)


In [3]:
# if '__main__' == __name__:

FOLDER_PATH = 'data/최종/'

# 원본 PATH
PATH1 = '재무상태표/'
PATH2 = '포괄손익계산서/'

# 전처리 PATH
PATH3 = '재무재표 전처리/'
PATH4 = '포괄손익계산서 전처리/'

# 파일 리스트화
file_list1 = os.listdir(FOLDER_PATH +'재무상태표/')
file_list2 = os.listdir(FOLDER_PATH +'포괄손익계산서/')

# 변수 지정
var_1_dic = {'자본총계': 4, '자산총계': 4, '매입채무': 13, '매출채권': 13, '부채총계': 4, '유동부채': 4, '유동자산': 4, '재고자산': 4}
var_2_dic = {'매출총이익': 5, '법인세비용차감': 15, '당기순이익': 10}

# 빈 데이터프레임 생성
bs_a = pd.DataFrame(columns=['date', 'company'])
cis_a = pd.DataFrame(columns=['date', 'company'])

for cmp_list1, cmp_list2 in zip(file_list1, file_list2):
    # 재무 상태표
    df = company_prerocessing.open_df(os.path.join(FOLDER_PATH + PATH1  + cmp_list1))
    df_bs_1 = company_prerocessing.bs_preprocessing(df, cmp_list1[:-13])
    company_prerocessing.save_df(os.path.join(FOLDER_PATH + PATH3), df_bs_1, cmp_list1[:-13])

    # 변수 전처리
    list1 = ['date', 'company']
    for var1, var1_ in var_1_dic.items():
        for bs in df_bs_1.columns:
            if var1 in bs:
                if var1_ >= len(bs):
                    list1.append(bs)
    print(df_bs_1[list1])
    # 재무상태표 데이터프레임 합치기
    bs_a = pd.concat([bs_a, df_bs_1[list1]])

    # 포괄 손익 계산서
    df = company_prerocessing.open_df(os.path.join(FOLDER_PATH + PATH2 + cmp_list2))
    df_cis_1 = company_prerocessing.cis_preprocessing(df, cmp_list2[:-15])
    company_prerocessing.save_df(os.path.join(FOLDER_PATH + PATH4), df_cis_1, cmp_list2[:-15])

    # 변수 전처리
    list2 = ['date', 'company']
    for var2, var2_ in var_2_dic.items():
        for cis in df_cis_1.columns:
            if var2 in cis:
                if var2_ >= len(cis):
                    list2.append(cis)
    print(df_cis_1[list2])
    # 포괄손익 계산서 데이터프레임 합치기
    cis_a = pd.concat([cis_a, df_cis_1[list2]])

# 인덱스 리셋 및 저장
bs_a.reset_index(inplace=True, drop=True)
cis_a.reset_index(inplace=True, drop=True)
company_prerocessing.save_df(os.path.join(FOLDER_PATH + PATH3), bs_a, '재무상태표 통합')
company_prerocessing.save_df(os.path.join(FOLDER_PATH + PATH4), cis_a, '포괄손익계산서 통합')


bs_preprocessing complete
       date company           자본총계            자산총계  매입채무 및 기타유동채무  \
0  20111231    금강제강  61279687814.0  194206670604.0  39110749325.0   
1  20101231    금강제강  72211501426.0  179481870466.0  41074511130.0   
2  20091231    금강제강  55248061746.0  120141443625.0  21023313155.0   

   매출채권 및 기타유동채권            부채총계            유동부채            유동자산  \
0  49725295614.0  132926982790.0  113985044750.0  100969321003.0   
1  61178350458.0  107270369040.0   85302556870.0   99806911954.0   
2  37786731906.0   64893381879.0   50998611676.0   57435150864.0   

            재고자산  
0  37436698391.0  
1  33343253080.0  
2  14166765495.0  
cis_preprocessing complete
       date company          매출총이익 법인세비용차감전순이익(손실)       당기순이익(손실)
0  20111231    금강제강    591907148.0  -13509435372.0  -11260872179.0
1  20101231    금강제강  13444476213.0    2073005833.0    1519307289.0
bs_preprocessing complete
       date company           자본총계            자산총계  매입채무 및 기타유동채무  \
0  20111231    배명금속  9529

In [50]:
result = pd.merge(bs_a, cis_a, on=['company', 'date'])

In [51]:
result['자본총계'].min()

'152837431616.0'

In [52]:
result

,date,company,자본총계,자산총계,매입채무 및 기타유동채무,매출채권 및 기타유동채권,부채총계,유동부채,유동자산,재고자산,매출총이익,법인세비용차감전순이익(손실),당기순이익(손실)
0,20111231,금강제강,61279687814.0,194206670604.0,39110749325.0,49725295614.0,132926982790.0,113985044750.0,100969321003.0,37436698391.0,591907148.0,-13509435372.0,-11260872179.0
1,20101231,금강제강,72211501426.0,179481870466.0,41074511130.0,61178350458.0,107270369040.0,85302556870.0,99806911954.0,33343253080.0,13444476213.0,2073005833.0,1519307289.0
2,20111231,배명금속,95296464290.0,208619609146.0,26367131866.0,30166438816.0,113323144856.0,106984768096.0,120559651206.0,66618482419.0,8510337046.0,376240588.0,627781200.0
3,20101231,배명금속,77745475250.0,179399062436.0,18536080027.0,26118974181.0,101653587186.0,95049265962.0,94315952312.0,55218399050.0,7700935418.0,-5482424803.0,-5245282081.0
4,20111231,풍림산업,152837431616.0,1374255960342.0,516846395394.0,759265274862.0,1221418528726.0,752955724323.0,919718310569.0,30002292372.0,2072700894.0,-41366351473.0,-35852563764.0
5,20101231,풍림산업,185149189773.0,1188801152279.0,371141947782.0,658773549500.0,1003651962506.0,954388334556.0,771447482160.0,34818379461.0,29044164067.0,-91069640922.0,-74624076085.0


In [53]:
'''
날짜 제외

'''

'\n날짜 제외\n\n'

In [54]:
import random as rnd

In [55]:
result.drop('date', inplace=True, axis=1)

In [47]:
result

,company,자본총계,자산총계,매입채무 및 기타유동채무,매출채권 및 기타유동채권,부채총계,유동부채,유동자산,재고자산,매출총이익,법인세비용차감전순이익(손실),당기순이익(손실)
0,금강제강,61279687814.0,194206670604.0,39110749325.0,49725295614.0,132926982790.0,113985044750.0,100969321003.0,37436698391.0,591907148.0,-13509435372.0,-11260872179.0
1,금강제강,72211501426.0,179481870466.0,41074511130.0,61178350458.0,107270369040.0,85302556870.0,99806911954.0,33343253080.0,13444476213.0,2073005833.0,1519307289.0
2,배명금속,95296464290.0,208619609146.0,26367131866.0,30166438816.0,113323144856.0,106984768096.0,120559651206.0,66618482419.0,8510337046.0,376240588.0,627781200.0
3,배명금속,77745475250.0,179399062436.0,18536080027.0,26118974181.0,101653587186.0,95049265962.0,94315952312.0,55218399050.0,7700935418.0,-5482424803.0,-5245282081.0
4,풍림산업,152837431616.0,1374255960342.0,516846395394.0,759265274862.0,1221418528726.0,752955724323.0,919718310569.0,30002292372.0,2072700894.0,-41366351473.0,-35852563764.0
5,풍림산업,185149189773.0,1188801152279.0,371141947782.0,658773549500.0,1003651962506.0,954388334556.0,771447482160.0,34818379461.0,29044164067.0,-91069640922.0,-74624076085.0


In [48]:
result.columns

Index(['company', '자본총계', '자산총계', '매입채무 및 기타유동채무', '매출채권 및 기타유동채권', '부채총계',
       '유동부채', '유동자산', '재고자산', '매출총이익', '법인세비용차감전순이익(손실)', '당기순이익(손실)'],
      dtype='object')

In [11]:
import string

In [12]:
string.ascii_uppercase

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [13]:
# asdf = []

# for i in range(1, len(result.columns)):
#     result[result.columns[i]] = pd.to_numeric(result[result.columns[i]])
#     print(rnd.randint(int(result[result.columns[i]].min()), int(result[result.columns[i]].max())))
#     asdf = rnd.randint(int(result[result.columns[i]].min()), int(result[result.columns[i]].max()))

# for j in string.ascii_uppercase:
#     asdf.append(j)
#     for k in string.ascii_uppercase:
#         asdf.append(rnd.randint(int(result[result.columns[i]].min()), int(result[result.columns[i]].max())))


In [14]:
asdf = []

In [15]:
for j in string.ascii_uppercase:
    print(j)
    asdf.append(j)
    for i in range(1, len(result.columns)):
        result[result.columns[i]] = pd.to_numeric(result[result.columns[i]])
#         print(rnd.randint(int(result[result.columns[i]].min()), int(result[result.columns[i]].max())))
        asdf.append(rnd.randint(int(result[result.columns[i]].min()), int(result[result.columns[i]].max())))

A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
U
V
W
X
Y
Z


In [16]:
asdf

['A',
 93880743862,
 1051204573037,
 291215987767,
 422230810851,
 865461174984,
 98034943786,
 772650389836,
 48964225915,
 28024515655,
 -3655524378,
 -22623319183,
 'B',
 82024629656,
 583009922827,
 61992157542,
 59487218175,
 538222686047,
 883817918350,
 662027847549,
 50263414216,
 16797714242,
 -22936493665,
 -72207718460,
 'C',
 65449787435,
 370731354450,
 207250180885,
 433088059554,
 650170493722,
 744600787050,
 322902509730,
 55301730781,
 18227782578,
 -80825622988,
 -41983075858,
 'D',
 76318378040,
 591946458469,
 482123033860,
 79767515924,
 817810965141,
 448945909923,
 904651226858,
 62598892246,
 9857742720,
 -13248093359,
 -49720050164,
 'E',
 114426441952,
 1204551542079,
 369845236690,
 454498208834,
 193236269123,
 166677962141,
 579415814701,
 39618360040,
 4439416710,
 -19254199642,
 -36697476119,
 'F',
 170680854400,
 748006032896,
 216285207468,
 167299791504,
 233572232688,
 692921117997,
 134188908137,
 53158342499,
 21710036785,
 -58922293390,
 -58184880

In [17]:
len(asdf)

312

In [148]:
12*26

312

In [18]:
import numpy as np

In [27]:
data = np.array(asdf)

In [28]:
data.shape

(312,)

In [31]:
data = data.reshape(-1,12)

In [32]:
data

array([['A', '93880743862', '1051204573037', '291215987767',
        '422230810851', '865461174984', '98034943786', '772650389836',
        '48964225915', '28024515655', '-3655524378', '-22623319183'],
       ['B', '82024629656', '583009922827', '61992157542', '59487218175',
        '538222686047', '883817918350', '662027847549', '50263414216',
        '16797714242', '-22936493665', '-72207718460'],
       ['C', '65449787435', '370731354450', '207250180885',
        '433088059554', '650170493722', '744600787050', '322902509730',
        '55301730781', '18227782578', '-80825622988', '-41983075858'],
       ['D', '76318378040', '591946458469', '482123033860',
        '79767515924', '817810965141', '448945909923', '904651226858',
        '62598892246', '9857742720', '-13248093359', '-49720050164'],
       ['E', '114426441952', '1204551542079', '369845236690',
        '454498208834', '193236269123', '166677962141', '579415814701',
        '39618360040', '4439416710', '-19254199642', '-3669

In [56]:
for i in range(6, len(string.ascii_uppercase)+6):
    result.loc[i] = data[i-6]

In [58]:
result

,company,자본총계,자산총계,매입채무 및 기타유동채무,매출채권 및 기타유동채권,부채총계,유동부채,유동자산,재고자산,매출총이익,법인세비용차감전순이익(손실),당기순이익(손실)
0,금강제강,61279687814.0,194206670604.0,39110749325.0,49725295614.0,132926982790.0,113985044750.0,100969321003.0,37436698391.0,591907148.0,-13509435372.0,-11260872179.0
1,금강제강,72211501426.0,179481870466.0,41074511130.0,61178350458.0,107270369040.0,85302556870.0,99806911954.0,33343253080.0,13444476213.0,2073005833.0,1519307289.0
2,배명금속,95296464290.0,208619609146.0,26367131866.0,30166438816.0,113323144856.0,106984768096.0,120559651206.0,66618482419.0,8510337046.0,376240588.0,627781200.0
3,배명금속,77745475250.0,179399062436.0,18536080027.0,26118974181.0,101653587186.0,95049265962.0,94315952312.0,55218399050.0,7700935418.0,-5482424803.0,-5245282081.0
4,풍림산업,152837431616.0,1374255960342.0,516846395394.0,759265274862.0,1221418528726.0,752955724323.0,919718310569.0,30002292372.0,2072700894.0,-41366351473.0,-35852563764.0
5,풍림산업,185149189773.0,1188801152279.0,371141947782.0,658773549500.0,1003651962506.0,954388334556.0,771447482160.0,34818379461.0,29044164067.0,-91069640922.0,-74624076085.0
6,A,93880743862,1051204573037,291215987767,422230810851,865461174984,98034943786,772650389836,48964225915,28024515655,-3655524378,-22623319183
7,B,82024629656,583009922827,61992157542,59487218175,538222686047,883817918350,662027847549,50263414216,16797714242,-22936493665,-72207718460
8,C,65449787435,370731354450,207250180885,433088059554,650170493722,744600787050,322902509730,55301730781,18227782578,-80825622988,-41983075858
9,D,76318378040,591946458469,482123033860,79767515924,817810965141,448945909923,904651226858,62598892246,9857742720,-13248093359,-49720050164


In [57]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 0 to 31
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   company          32 non-null     object
 1   자본총계             32 non-null     object
 2   자산총계             32 non-null     object
 3   매입채무 및 기타유동채무    32 non-null     object
 4   매출채권 및 기타유동채권    32 non-null     object
 5   부채총계             32 non-null     object
 6   유동부채             32 non-null     object
 7   유동자산             32 non-null     object
 8   재고자산             32 non-null     object
 9   매출총이익            32 non-null     object
 10  법인세비용차감전순이익(손실)  32 non-null     object
 11  당기순이익(손실)        32 non-null     object
dtypes: object(12)
memory usage: 3.2+ KB


In [13]:
var_1_dic = {'자본총계': 4, '자산총계': 4, '매입채무': 13, '매출채권': 13, '부채총계': 4, '유동부채': 4, '유동자산': 4, '재고자산': 4}
var_2_dic = {'매출총이익': 5, '법인세비용차감': 15, '당기순이익': 10}

In [14]:
df = company_prerocessing.open_df(os.path.join(FOLDER_PATH + '재무상태표/' + file_list1[0]))
df_bs_1 = company_prerocessing.bs_preprocessing(df, cmp_list1[:-13])
company_prerocessing.save_df(os.path.join(FOLDER_PATH + '재무재표 전처리/'), df_bs_1, cmp_list1[:-13])

bs_preprocessing complete


In [15]:
list1 = ['date', 'company']
for var1, var1_ in var_1_dic.items():
    for bs in df_bs_1.columns:
        if var1 in bs:
            if var1_ == len(bs):
                print(bs)
                list1.append(bs)
# print(df_bs_1[list1])
# bs_a = pd.concat([bs_a, df_bs_1[list1]])

자본총계
자산총계
매입채무 및 기타유동채무
매출채권 및 기타유동채권
부채총계
유동부채
유동자산
재고자산


In [16]:
len('매출채권 및 기타유동채권')

13